# SAI - I 트리케라톱스
실습과제

- 회귀 문제를 다뤄보겠음
- 데이터셋은 여러 개의 feature가 있음

In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [2]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


13가지의 특성이 존재함. 13가지는 서로 평균, 분산이 모두 다른 정보. 안정적인 학습을 위해선 normalization이 필요하다.

In [ ]:
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= staticmethod

test_data -=mean
test_data /= std

여기서 주의깊게 봐야 하는 부분은 **train data의 평균과 분산을 test data를 normalization 시킬 때도 동일하게 사용**했다는 점이다.

test data를 갖고는 어떠한 연산도 해선 안 되기 때문이다.

In [3]:
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
                        input_shape = (train_data.shape[1], )))
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(1))
  
  model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
  return model

모델을 여러 번 새로 갈아엎을 것이기 때문에 선언부를 함수로 만들어 준다.
훈련 데이터의 개수가 적을 수록 과대적합이 쉽게 일어나는 편이다. 따라서 작은 모델을 쓰는 편이 좋다.
- MSE : Mean Square Error. 회귀 문제에서 자주 쓰이는 편이다.
- MAE : Mean Absolute Error. 모니터링을 위해 쓰는 지표인데, 예측과 타깃 사이 거리의 절대값을 의미한다.

## K-겹 검증
K-fold cross-validation

모델을 평가하기 위해서 훈련 세트와 검증 세트로 나누어야 하는데, 데이터가 적어서 검증 세트도 작아진다. 그래서 검증 세트를 어떻게 나누었느냐에 따라 성능이 크게 달라지기도 한다. 이를 막기 위해 있는 방법이 K겹 검증임.

쉽게 말하면 그냥 K조각 만큼 데이터를 분할하고 한 조각을 평가 세트로, 나머지 조각들을 훈련 세트로 삼는 것이다. 그리고 촤종 점수는 그렇게 한 조각씩 선택해서 얻은 검증 점수의 평균을 취한다.

In [6]:
# from sklearn.model_selection import KFold # 교재에선 그냥 생으로 구현했는데, 한 번 외부 패키지 찾아 봤다.

In [8]:
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print("처리중인 폴드: {}".format(i))
  val_data = train_data[ i * num_val_samples: (i+1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]

  partial_train_data = np.concatenate(
      [train_data[:i*num_val_samples],
       train_data[(i+1)*num_val_samples:]],
       axis = 0)
  partial_train_targets = np.concatenate(
      [train_targets[:i*num_val_samples],
       train_targets[(i+1)*num_val_samples:]],
       axis = 0)
  
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
            epochs = num_epochs, batch_size = 1, verbose = 0) # verbose = 0 이면 훈련 과정이 출력되지 않음
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0 )
  all_scores.append(val_mae)

처리중인 폴드: 0
처리중인 폴드: 1
처리중인 폴드: 2
처리중인 폴드: 3


In [9]:
print(all_scores)

[3.24340558052063, 2.8697080612182617, 3.2149622440338135, 3.5428271293640137]


In [10]:
np.mean(all_scores)

3.2177257537841797

그 다음 내용은.. 솔직히 무슨 내용인지 잘 모르겠음. 